# Projet : Forage de données IFT870

# Augmentaion de la région d'intérêt

Ce notebook contiendra des traces de la méthode utilisée pour augmenter la région d'intérêt de la tumeur.

### Installation des dépendances

In [4]:
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
from pymatreader import read_mat


# On se demande pourquoi pip install en windows powershell ne l'installe pas.
# !pip install pymatreader 

### Importation de données

In [55]:
nb_images = 3064

class_names = ['meningiome', 'gliome', 'pituitaire']

def load_data(type, nb_images=3064):
    data = [ [], [], [] ] # index : meningioma = 0, glioma = 1, pituitary = 2
    if type == 'mat':
        for i in range(1, nb_images):
            dir = "dataset/mat/" + str(i) + ".mat"
            mat_file = read_mat(dir)
            classe = int(mat_file["cjdata"]["label"]-1) # -1 car les indices commencent a 1 dans les fichiers mat
            data[classe].append(mat_file)
        return data
    
    if type == 'png':
        class_nb = [1, 2, 3]    
        for i in range(len(class_nb)):
            dir = "dataset/png/" + str(class_nb[i])
            for path in os.listdir(dir):
                # print(dir)
                # print(path)
                data[i].append(io.imread(dir + "/" + path))
        return data


mat_data = load_data(type='mat')
png_data = load_data(type='png', nb_images=2)
